In [1]:
import xarray as xr

In [2]:
nc_file = "data/gdas.t00z.sfcanl.nc"
# 3. Open as a single concatenated Dataset
ds_sfcanl = xr.open_dataset(nc_file)

In [3]:
ds_sfcanl

<xarray.Dataset> Size: 1GB
Dimensions:  (grid_xt: 3072, grid_yt: 1536, time: 1)
Coordinates:
  * grid_xt  (grid_xt) float64 25kB 0.0 0.1172 0.2344 ... 359.6 359.8 359.9
  * grid_yt  (grid_yt) float64 12kB 89.91 89.79 89.68 ... -89.68 -89.79 -89.91
  * time     (time) object 8B 2025-06-05 00:00:00
Data variables: (12/62)
    lon      (grid_yt, grid_xt) float64 38MB ...
    lat      (grid_yt, grid_xt) float64 38MB ...
    alnsf    (time, grid_yt, grid_xt) float32 19MB ...
    alnwf    (time, grid_yt, grid_xt) float32 19MB ...
    alvsf    (time, grid_yt, grid_xt) float32 19MB ...
    alvwf    (time, grid_yt, grid_xt) float32 19MB ...
    ...       ...
    xtts     (time, grid_yt, grid_xt) float32 19MB ...
    xu       (time, grid_yt, grid_xt) float32 19MB ...
    xv       (time, grid_yt, grid_xt) float32 19MB ...
    xz       (time, grid_yt, grid_xt) float32 19MB ...
    xzts     (time, grid_yt, grid_xt) float32 19MB ...
    zc       (time, grid_yt, grid_xt) float32 19MB ...
Attributes:
    nsoil:    4
    source:   FV3GFS
    grid:     gaussian
    im:       3072
    jm:       1536